In [2]:
import os
import argparse
import math
import torch
import torchvision
from torchvision.transforms import ToTensor, Compose, Normalize
from tqdm import tqdm
import scipy
import os
import matplotlib.pyplot as plt
from models.mae import *
from utils import setup_seed
from torch.utils.data import DataLoader, TensorDataset
import time

/home/ronald/anaconda3/envs/new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
V0_tensor = torch.tensor(np.load('Data/V/V_EIM0.npy'))
V1_tensor = torch.tensor(np.load('Data/V/V_EIM1.npy'))
V2_tensor = torch.tensor(np.load('Data/V/V_EIM2.npy'))
V3_tensor = torch.tensor(np.load('Data/V/V_EIM3.npy'))
V4_tensor = torch.tensor(np.load('Data/V/V_EIM4.npy'))

GT0_tensor = torch.tensor(np.load('Data/GT/GT0.npy'))
GT1_tensor = torch.tensor(np.load('Data/GT/GT1.npy'))
GT2_tensor = torch.tensor(np.load('Data/GT/GT2.npy'))
GT3_tensor = torch.tensor(np.load('Data/GT/GT3.npy'))
GT4_tensor = torch.tensor(np.load('Data/GT/GT4.npy'))

check numpy data type

In [7]:
print(V0_tensor.dtype)
print(np.load('Data/V/V_EIM0.npy').dtype)

torch.float32
float32


check mat data type

In [11]:
V0_mat = scipy.io.loadmat('Data/V/V_EIM0.mat')['V_EIM0']
print(V0_mat.dtype)

float64


### mat2npy in 32*32: Bicubic interpolation

In [20]:
import torch
import numpy as np
import scipy.io
from scipy.ndimage import zoom

def mat2tensor(V_mat):
    # 假设 V_mat 是一个已经加载的、形状为 (11520, 16, 16) 的双精度矩阵
    # 返回值是一个形状为 (11520, 32, 32) 的双精度矩阵
    V_resized = zoom(V_mat, (1, 2, 2), order=3)  # order=3 对应双三次插值
    return torch.from_numpy(V_resized).float().unsqueeze(1)

V0_mat = scipy.io.loadmat('Data/V/V_EIM0.mat')['V_EIM0']
V1_mat = scipy.io.loadmat('Data/V/V_EIM1.mat')['V_EIM1']
V2_mat = scipy.io.loadmat('Data/V/V_EIM2.mat')['V_EIM2']
V3_mat = scipy.io.loadmat('Data/V/V_EIM3.mat')['V_EIM3']
V4_mat = scipy.io.loadmat('Data/V/V_EIM4.mat')['V_EIM4']
V0_tensor = mat2tensor(V0_mat)
V1_tensor = mat2tensor(V1_mat)
V2_tensor = mat2tensor(V2_mat)
V3_tensor = mat2tensor(V3_mat)
V4_tensor = mat2tensor(V4_mat)
print(V0_tensor.shape)
# print(V0_tensor)
np.save('Data/V/V_EIM0_32.npy', V0_tensor)
np.save('Data/V/V_EIM1_32.npy', V1_tensor)
np.save('Data/V/V_EIM2_32.npy', V2_tensor)
np.save('Data/V/V_EIM3_32.npy', V3_tensor)
np.save('Data/V/V_EIM4_32.npy', V4_tensor)

New tensor shape: torch.Size([100000, 1, 32, 32])
New tensor shape: torch.Size([100000, 1, 32, 32])
New tensor shape: torch.Size([100000, 1, 32, 32])
New tensor shape: torch.Size([100000, 1, 32, 32])
torch.Size([11520, 1, 32, 32])


### mat2npy in 64*64: Bicubic interpolation

In [ ]:
import torch
import numpy as np
import scipy.io
from scipy.ndimage import zoom

def mat2tensor(V_mat, dtype=torch.float64):
    """
    将给定的numpy矩阵转换为PyTorch张量。
    
    Args:
        V_mat (np.ndarray): 形状为 (N, 16, 16) 的numpy数组。
        dtype (torch.dtype, optional): 目标张量的数据类型。默认为torch.float64。
        
    Returns:
        torch.Tensor: 形状为 (N, 1, 64, 64) 的PyTorch张量。
    """
    V_resized = zoom(V_mat, (1, 4, 4), order=3)  # order=3 对应双三次插值
    return torch.from_numpy(V_resized).to(dtype).unsqueeze(1)

V0_mat = scipy.io.loadmat('Data/V/V_EIM0.mat')['V_EIM0']
V1_mat = scipy.io.loadmat('Data/V/V_EIM1.mat')['V_EIM1']
V2_mat = scipy.io.loadmat('Data/V/V_EIM2.mat')['V_EIM2']
V3_mat = scipy.io.loadmat('Data/V/V_EIM3.mat')['V_EIM3']
V4_mat = scipy.io.loadmat('Data/V/V_EIM4.mat')['V_EIM4']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

V0_tensor = mat2tensor(V0_mat).to(device)
V1_tensor = mat2tensor(V1_mat).to(device)
V2_tensor = mat2tensor(V2_mat).to(device)
V3_tensor = mat2tensor(V3_mat).to(device)
V4_tensor = mat2tensor(V4_mat).to(device)
print(V0_tensor.shape)
# print(V0_tensor)
np.save('Data/V/V_EIM0_64.npy', V0_tensor.cpu().numpy())
np.save('Data/V/V_EIM1_64.npy', V1_tensor.cpu().numpy())
np.save('Data/V/V_EIM2_64.npy', V2_tensor.cpu().numpy())
np.save('Data/V/V_EIM3_64.npy', V3_tensor.cpu().numpy())
np.save('Data/V/V_EIM4_64.npy', V4_tensor.cpu().numpy())